# Avaliação de Datasets problemáticos

In [247]:
import os

import ipywidgets as widgets
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, StratifiedKFold
import matplotlib.pyplot as plt

In [243]:
def get_dataset_df(dataset_name):
    return pd.read_csv(os.path.join(DATA_DIR, 'csv', dataset_name + '.csv'))


## `ozone-eighthr`

In [252]:
dataset_name = "ozone-eighthr"
df =  get_dataset_df(dataset_name)
df.describe()

,remainder__V1,remainder__V2,remainder__V3,remainder__V4,remainder__V5,remainder__V6,remainder__V7,remainder__V8,remainder__V9,remainder__V10,...,remainder__V64,remainder__V65,remainder__V66,remainder__V67,remainder__V68,remainder__V69,remainder__V70,remainder__V71,remainder__V72,remainder__Class
count,2534.000000,2534.000000,2534.000000,2534.000000,2534.000000,2534.000000,2534.000000,2534.000000,2534.000000,2534.000000,...,2534.000000,2534.000000,2534.000000,2534.000000,2534.000000,2534.000000,2534.000000,2534.000000,2534.000000,2534.000000
mean,1.640179,1.586351,1.545580,1.526405,1.522624,1.542417,1.637896,2.047127,2.539037,2.847708,...,0.304716,9.872418,0.830116,5818.821222,10.511051,37.388335,10164.198442,-0.119949,0.372026,0.063141
std,1.194568,1.191320,1.165651,1.133908,1.127523,1.102557,1.092811,1.092315,1.115448,1.150067,...,0.242858,9.127381,7.043762,77.410110,20.154266,10.949074,51.428116,34.693349,1.317438,0.243265
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.100000,...,0.010000,-14.920000,-25.990000,5480.000000,-56.700000,-10.100000,9975.000000,-135.000000,0.000000,0.000000
25%,0.800000,0.700000,0.700000,0.700000,0.700000,0.700000,0.800000,1.300000,1.800000,2.100000,...,0.100000,3.130000,-3.340000,5771.250000,-2.437500,33.200000,10130.000000,-20.000000,0.000000,0.000000
50%,1.600000,1.500000,1.400000,1.400000,1.400000,1.400000,1.600000,2.047127,2.539037,2.847708,...,0.250000,9.872418,0.830116,5825.000000,13.000000,40.500000,10160.000000,-0.119949,0.000000,0.000000
75%,2.200000,2.100000,2.100000,2.075000,2.000000,2.000000,2.100000,2.600000,3.200000,3.500000,...,0.460000,16.097500,4.380000,5880.000000,27.737500,44.950000,10190.000000,15.000000,0.050000,0.000000
max,7.500000,7.700000,7.100000,7.300000,7.200000,7.400000,7.400000,7.500000,9.200000,8.500000,...,1.000000,42.360000,30.420000,5965.000000,42.050000,59.150000,10350.000000,140.000000,20.650000,1.000000


In [269]:
def test_passive_svm(dataset_name):
    test_passive(get_dataset_df(dataset_name), SVC())
    
widgets.interact(test_passive_svm, dataset_name=widgets.Select(options=dataset_names, description="Dataset:"))

interactive(children=(Select(description='Dataset:', options=('abalone-3class', 'artificial-characters', 'auto…

<function __main__.test_passive_svm(dataset_name)>

Dataset extremamente desbalanceado

remainder__Class
0    2374
1     160
Name: count, dtype: int64

## Analizando demais Datasets

In [46]:
dataset_names = ! cat $DATA_DIR/datasets.txt

In [160]:
def plot_results(dataset_name, learner, ax):
    
    ax.set_title(dataset_name)
    
    dataset_name+='#'
    random_file, = !ls $RESULT_DIR | grep $dataset_name | grep $learner | grep 'random'
    margin_file, = !ls $RESULT_DIR | grep $dataset_name | grep $learner | grep 'margin'

    rand_df = pd.read_csv(os.path.join(RESULT_DIR,random_file), index_col=0)
    margin_df = pd.read_csv(os.path.join(RESULT_DIR, margin_file), index_col=0)



    ax.plot(rand_df.mean(axis=1), color="firebrick")

    for _, column in margin_df.items():
        ax.plot(column, linestyle='--', alpha=0.2, color="steelblue")
    
    ax.plot(margin_df.mean(axis=1), color="steelblue")

    ax.grid(True)
    ax.set_ybound(-1,1)

## Análise para SVM

In [161]:
def plot_svm(dataset_name):
    fig, ax = plt.subplots()
    plot_results(dataset_name, "SVC", ax)
    
widgets.interact(plot_svm,
                 dataset_name=widgets.SelectionSlider(options=dataset_names,
                                                      description="Dataset:"),
                ax=ax)

interactive(children=(SelectionSlider(description='Dataset:', options=('abalone-3class', 'artificial-character…

<function __main__.plot_svm(dataset_name)>

## Análise Geral

In [220]:
def plot_general(dataset_name):
    fig, axs = plt.subplots(2, 2)
    leaners = ["SVC", "5NN", "DecisionTree", "GaussianNB"]
    
    
    for learner, ax in zip(leaners, axs.flatten()):
        plot_results(dataset_name, learner, ax)
        ax.set_title(learner)
    
    
    df = pd.read_csv(os.path.join(DATA_DIR, 'csv', dataset_name + '.csv'))
    df_classes = df.iloc[:, -1]
    
    ent = entropy(df_classes.value_counts()/len(df_classes))
    n_labeled = (100/len(df_classes))
    
    fig.suptitle(f"{dataset_name}", y=1.02, fontweight="bold") 
    fig.text(0.5, 0.95,f"Entropia:{ent:.2f}, Porporção dados Anotados:{n_labeled:.2f}", ha="center")
    fig.tight_layout()
    
widgets.interact(plot_general,
                 dataset_name=widgets.SelectionSlider(options=dataset_names,
                                                      description="Dataset:"),
                 ax=ax)


interactive(children=(SelectionSlider(description='Dataset:', options=('abalone-3class', 'artificial-character…

<function __main__.plot_general(dataset_name)>

In [185]:
df = pd.read_csv(os.path.join(DATA_DIR, 'csv', dataset_names[0] + '.csv'))
y = df.iloc[:, -1]
print(100/len(y))
entropy(y.value_counts()/len(y))

0.023940627244433806


1.0979199138101299

In [185]:
df = pd.read_csv(os.path.join(DATA_DIR, 'csv', dataset_names[0] + '.csv'))


0.023940627244433806


1.0979199138101299

In [180]:
from scipy.stats import entropy